# 🚕 NYC Taxi Streaming Demo - Databricks Free Edition (SEM DBFS)

✨ **Compatível com Databricks Free Edition - Sem DBFS**

Este notebook demonstra **Structured Streaming** usando o dataset público **NYC Taxi** disponível nativamente no Databricks.

## 🎯 Objetivos da Demonstração
- Processar dados reais de táxis de NYC com streaming
- Usar **maxFilesPerTrigger e maxBytesPerTrigger** para controlar velocidade de processamento
- Demonstrar agregações em tempo real (viagens por região, receita por tipo de pagamento)
- Salvar resultados **diretamente em Delta Tables** (Unity Catalog)
- Visualizar métricas batch por batch com foreachBatch

## 📊 Dataset: samples.nyctaxi.trips
- **Fonte:** Unity Catalog do Databricks (já disponível!)
- **Registros:** Milhões de viagens de táxi em NYC
- **Período:** Dados históricos de táxis amarelos e verdes
- **Campos:** pickup/dropoff datetime, locations, distances, fares, payment types

## ✅ Vantagens desta abordagem
- ❌ Sem Kafka externo (Free Edition tem rede restrita)
- ❌ **Sem DBFS** (Public DBFS root está desabilitado)
- ❌ Sem FileStore (também desabilitado)
- ✅ Usa dados REAIS (não sintéticos)
- ✅ 100% nativo do Databricks com Delta Tables
- ✅ Zero configuração de rede ou filesystem externo
- ✅ Perfeito para demonstrações educacionais

## 1️⃣ Setup e Configuração (SEM DBFS)

Vamos configurar o streaming **diretamente** da tabela Unity Catalog, salvando em Delta Tables gerenciadas.

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# Configurações do streaming - TUDO GERENCIADO, SEM DBFS
TABLE_SOURCE = "samples.nyctaxi.trips"
OUTPUT_TABLE = "default.nyctaxi_streaming_results"  # Tabela Delta gerenciada no Catalog

print("🚕 NYC TAXI STREAMING DEMO (SEM DBFS)")
print("=" * 80)
print(f"📊 Fonte:      {TABLE_SOURCE} (Unity Catalog)")
print(f"💾 Destino:    {OUTPUT_TABLE} (Delta Table Gerenciada)")
print("=" * 80)
print("\n✅ Abordagem:")
print("   • Ler diretamente de Unity Catalog com readStream")
print("   • Processar batches controlados por maxFilesPerTrigger e maxBytesPerTrigger")
print("   • Salvar em Delta Table gerenciada (sem DBFS)")
print("   • Visualizar métricas com foreachBatch")
print("=" * 80)

## 2️⃣ Explorar Dados da Tabela NYC Taxi

Vamos primeiro entender os dados disponíveis na tabela Unity Catalog.

In [ ]:
print("📊 Explorando samples.nyctaxi.trips...")

# Ler amostra da tabela NYC Taxi
nyctaxi_sample = spark.table(TABLE_SOURCE).limit(10)

print("\n📋 Schema dos dados:")
nyctaxi_sample.printSchema()

print("\n🔍 Amostra dos primeiros registros:")
nyctaxi_sample.show(5, truncate=False)

# Estatísticas rápidas
print("\n📈 Estatísticas rápidas (sample):")
nyctaxi_sample.select(
    count("*").alias("registros"),
    min("tpep_pickup_datetime").alias("primeira_viagem"),
    max("tpep_pickup_datetime").alias("ultima_viagem"),
    avg("trip_distance").alias("distancia_media"),
    avg("total_amount").alias("valor_medio")
).show()

## 3️⃣ Configurar Structured Streaming da Tabela Delta

Vamos criar um streaming **direto da tabela** usando `maxFilesPerTrigger` e `maxBytesPerTrigger` para controlar a velocidade de processamento.

In [ ]:
# Criar streaming DataFrame diretamente da tabela Unity Catalog
streaming_df = spark.readStream \
    .option("maxFilesPerTrigger", 2) \
    .option("maxBytesPerTrigger", "5m") \
    .table(TABLE_SOURCE)

print("✅ Streaming configurado!")
print(f"⚙️  Fonte: {TABLE_SOURCE}")
print(f"⚙️  maxFilesPerTrigger: 2 (processa até 2 arquivos Delta por batch)")
print(f"⚙️  maxBytesPerTrigger: 5m (máximo 5MB por batch)")
print(f"📊 É streaming? {streaming_df.isStreaming}")
print("\n📋 Schema:")
streaming_df.printSchema()

print("\n💡 Nota: O streaming lerá incrementalmente os arquivos da tabela Delta,")
print("   processando alguns arquivos por batch para demonstração visual.")

## 4️⃣ Enriquecer e Agregar Dados

Adicionamos colunas de análise e criamos agregações por região e tipo de pagamento.

In [ ]:
# Enriquecer dados
enriched_df = streaming_df \
    .withColumn("processing_time", current_timestamp()) \
    .withColumn("pickup_date", to_date(col("tpep_pickup_datetime"))) \
    .withColumn("pickup_hour", hour(col("tpep_pickup_datetime"))) \
    .withColumn("trip_duration_minutes", 
                (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60) \
    .withColumn("payment_type_desc", 
                when(col("payment_type") == 1, "Credit Card")
                .when(col("payment_type") == 2, "Cash")
                .when(col("payment_type") == 3, "No Charge")
                .when(col("payment_type") == 4, "Dispute")
                .otherwise("Unknown"))

# Agregações por localização de pickup e tipo de pagamento
aggregated_df = enriched_df \
    .groupBy("pickup_location_id", "payment_type_desc", "pickup_date") \
    .agg(
        count("*").alias("total_trips"),
        sum("trip_distance").alias("total_distance"),
        sum("total_amount").alias("total_revenue"),
        avg("total_amount").alias("avg_fare"),
        avg("trip_distance").alias("avg_distance"),
        avg("trip_duration_minutes").alias("avg_duration_min"),
        min("tpep_pickup_datetime").alias("first_trip"),
        max("tpep_pickup_datetime").alias("last_trip")
    )

print("✅ Pipeline de agregação criado!")
print("\n📊 Métricas calculadas:")
print("  • Total de viagens")
print("  • Distância total percorrida")
print("  • Receita total")
print("  • Ticket médio (fare)")
print("  • Distância média")
print("  • Duração média (minutos)")

## 5️⃣ foreachBatch - Processar e Visualizar Cada Batch

Usamos `foreachBatch` para ver o processamento **batch por batch** em tempo real!

In [ ]:
# Estatísticas acumuladas
batch_stats = {
    'count': 0,
    'total_trips': 0,
    'total_revenue': 0,
    'total_distance': 0
}

def process_batch(batch_df, batch_id):
    """Processa cada batch do streaming"""
    batch_stats['count'] += 1
    
    print("\n" + "=" * 100)
    print(f"📦 BATCH #{batch_stats['count']} | Batch ID: {batch_id}")
    print("=" * 100)
    
    if batch_df.count() == 0:
        print("⚠️  Batch vazio - pulando")
        return
    
    # Estatísticas do batch
    stats = batch_df.agg(
        sum("total_trips").alias("trips"),
        sum("total_revenue").alias("revenue"),
        sum("total_distance").alias("distance")
    ).collect()[0]
    
    batch_stats['total_trips'] += stats.trips if stats.trips else 0
    batch_stats['total_revenue'] += stats.revenue if stats.revenue else 0
    batch_stats['total_distance'] += stats.distance if stats.distance else 0
    
    print(f"\n📊 ESTATÍSTICAS DO BATCH:")
    print(f"   🚕 Viagens: {stats.trips:,}")
    print(f"   💰 Receita: ${stats.revenue:,.2f}")
    print(f"   📏 Distância: {stats.distance:,.2f} milhas")
    
    # Top 5 localizações por receita
    print(f"\n🏆 TOP 5 LOCALIZAÇÕES (Receita):")
    top_locations = batch_df.orderBy(col("total_revenue").desc()).limit(5)
    top_locations.select(
        "pickup_location_id",
        "total_trips",
        "total_revenue",
        "avg_fare"
    ).show(truncate=False)
    
    # Distribuição por tipo de pagamento
    print(f"💳 DISTRIBUIÇÃO POR TIPO DE PAGAMENTO:")
    payment_dist = batch_df.groupBy("payment_type_desc") \
        .agg(
            sum("total_trips").alias("trips"),
            sum("total_revenue").alias("revenue")
        ) \
        .orderBy(col("revenue").desc())
    payment_dist.show(truncate=False)
    
    # Totais acumulados
    print(f"\n📈 TOTAIS ACUMULADOS:")
    print(f"   📦 Batches processados: {batch_stats['count']}")
    print(f"   🚕 Viagens: {batch_stats['total_trips']:,}")
    print(f"   💰 Receita: ${batch_stats['total_revenue']:,.2f}")
    print(f"   📏 Distância: {batch_stats['total_distance']:,.2f} milhas")
    if batch_stats['total_trips'] > 0:
        print(f"   🎯 Receita Média/Viagem: ${batch_stats['total_revenue']/batch_stats['total_trips']:,.2f}")

print("🔧 Função de processamento definida!")

## 6️⃣ EXECUTAR Streaming com foreachBatch

**Execute esta célula e observe o processamento batch por batch!**

O streaming irá:
1. Processar dados incrementalmente da tabela Delta
2. Mostrar métricas de cada batch
3. Acumular estatísticas totais
4. Executar por 60 segundos (ou até processar todos os dados)

In [ ]:
print("🚀 Iniciando streaming...\n")

# Iniciar streaming com foreachBatch
query = aggregated_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("complete") \
    .trigger(processingTime="5 seconds") \
    .start()

print("✅ Streaming iniciado!")
print("⏱️  Executará por 60 segundos...")
print("\n👀 Observe as métricas sendo atualizadas abaixo:\n")

## 7️⃣ Aguardar e Parar o Streaming

Deixe o streaming executar por 60 segundos para ver vários batches, depois pare.

In [ ]:
# Aguardar 60 segundos
print("⏳ Aguardando 60 segundos...")
time.sleep(60)

# Parar o streaming
query.stop()
print("\n🛑 Streaming parado!")

print("\n" + "=" * 100)
print("📊 RESUMO FINAL")
print("=" * 100)
print(f"📦 Total de batches processados: {batch_stats['count']}")
print(f"🚕 Total de viagens: {batch_stats['total_trips']:,}")
print(f"💰 Receita total: ${batch_stats['total_revenue']:,.2f}")
print(f"📏 Distância total: {batch_stats['total_distance']:,.2f} milhas")
if batch_stats['total_trips'] > 0:
    print(f"🎯 Receita média por viagem: ${batch_stats['total_revenue']/batch_stats['total_trips']:,.2f}")
    print(f"📍 Distância média por viagem: {batch_stats['total_distance']/batch_stats['total_trips']:,.2f} milhas")
print("=" * 100)

## 8️⃣ ALTERNATIVA: Salvar em Delta Table Gerenciada

Em vez de usar `foreachBatch`, podemos salvar diretamente em uma **Delta Table gerenciada** no Catalog.

**Vantagens:**
- Dados persistidos automaticamente
- Pode consultar com SQL
- Suporta ACID transactions
- Gerenciado pelo Unity Catalog

In [ ]:
print(f"💾 Salvando resultados em Delta Table: {OUTPUT_TABLE}")

# Limpar tabela anterior se existir
try:
    spark.sql(f"DROP TABLE IF EXISTS {OUTPUT_TABLE}")
    print(f"🧹 Tabela anterior removida")
except:
    pass

# Iniciar streaming salvando em Delta Table
query_delta = aggregated_df.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", f"/tmp/checkpoint_{int(time.time())}") \
    .trigger(processingTime="10 seconds") \
    .toTable(OUTPUT_TABLE)

print(f"✅ Streaming para Delta Table iniciado!")
print(f"📊 Tabela: {OUTPUT_TABLE}")
print(f"⏱️  Trigger: 10 segundos")
print(f"\n💡 A tabela será atualizada automaticamente a cada 10 segundos")

## 9️⃣ Aguardar Processamento da Delta Table

In [ ]:
# Aguardar 60 segundos para processar dados
print("⏳ Aguardando 60 segundos para processar dados...")
time.sleep(60)

# Parar o streaming
query_delta.stop()
print("🛑 Streaming para Delta Table parado!")

## 🔟 Consultar Resultados da Delta Table

Agora podemos consultar os resultados salvos na Delta Table!

In [ ]:
print(f"📊 Consultando resultados de {OUTPUT_TABLE}...\n")

# Ler a tabela
results_df = spark.table(OUTPUT_TABLE)

print(f"✅ Total de registros salvos: {results_df.count():,}\n")

# Top 10 localizações por receita
print("🏆 TOP 10 LOCALIZAÇÕES (Receita Total):")
results_df.orderBy(col("total_revenue").desc()).limit(10).show(truncate=False)

# Estatísticas gerais
print("\n📈 ESTATÍSTICAS GERAIS:")
totals = results_df.agg(
    sum("total_trips").alias("total_viagens"),
    sum("total_revenue").alias("receita_total"),
    sum("total_distance").alias("distancia_total"),
    avg("avg_fare").alias("ticket_medio"),
    avg("avg_distance").alias("distancia_media_viagem")
).collect()[0]

print(f"  🚕 Total de viagens: {totals.total_viagens:,}")
print(f"  💰 Receita total: ${totals.receita_total:,.2f}")
print(f"  📏 Distância total: {totals.distancia_total:,.2f} milhas")
print(f"  🎯 Ticket médio: ${totals.ticket_medio:,.2f}")
print(f"  📍 Distância média por viagem: {totals.distancia_media_viagem:,.2f} milhas")

## 1️⃣1️⃣ Consultar com SQL

Podemos usar SQL diretamente na tabela Delta!

In [ ]:
# Exemplo de consulta SQL
spark.sql(f"""
SELECT 
    payment_type_desc,
    SUM(total_trips) as viagens,
    SUM(total_revenue) as receita,
    AVG(avg_fare) as ticket_medio
FROM {OUTPUT_TABLE}
GROUP BY payment_type_desc
ORDER BY receita DESC
""").show(truncate=False)

## 1️⃣2️⃣ Cleanup (Opcional)

Remova a tabela de testes se desejar.

In [ ]:
# Descomentar para executar cleanup
# spark.sql(f"DROP TABLE IF EXISTS {OUTPUT_TABLE}")
# print(f"🧹 Tabela {OUTPUT_TABLE} removida")

print("ℹ️  Cleanup desabilitado. Descomente o código acima para remover a tabela.")

---

## 📚 Conceitos Demonstrados

### Structured Streaming
- **readStream**: Lê dados de forma incremental de tabelas Delta
- **maxFilesPerTrigger**: Controla quantos arquivos processar por batch
- **maxBytesPerTrigger**: Controla volume de dados por batch
- **writeStream**: Salva resultados do streaming
- **foreachBatch**: Função customizada para processar cada batch
- **outputMode=complete**: Retorna resultados completos das agregações

### Delta Lake
- **Delta Tables gerenciadas**: Tabelas no Unity Catalog
- **ACID transactions**: Garantia de consistência
- **Time travel**: Histórico de versões (não explorado aqui)
- **Schema enforcement**: Validação automática de schema

### Unity Catalog
- **samples.nyctaxi.trips**: Dataset público nativo
- **default schema**: Schema padrão para tabelas
- **Managed tables**: Metadados e dados gerenciados

---

## ✅ Checklist de Execução

1. ✅ Configurar variáveis (TABLE_SOURCE, OUTPUT_TABLE)
2. ✅ Explorar dados da tabela NYC Taxi
3. ✅ Configurar streaming com maxFilesPerTrigger
4. ✅ Criar pipeline de agregações
5. ✅ Executar com foreachBatch (Opção 1)
   - Ver métricas batch por batch
   - Aguardar 60 segundos
   - Parar streaming
6. ✅ OU Salvar em Delta Table (Opção 2)
   - Iniciar streaming para Delta
   - Aguardar 60 segundos
   - Parar streaming
   - Consultar resultados
7. ✅ Consultar com SQL
8. ✅ Cleanup (opcional)

---

## 🎓 Para Professores

Este notebook é ideal para demonstrar:

1. **Processamento Incremental**: Como o Spark processa dados em batches
2. **Agregações em Streaming**: GroupBy com funções de agregação
3. **Delta Lake**: Vantagens de usar Delta vs Parquet
4. **Unity Catalog**: Governança e organização de dados
5. **Controle de Fluxo**: maxFilesPerTrigger e maxBytesPerTrigger
6. **Visualização de Métricas**: foreachBatch para inspeção

**Sem necessidade de:**
- Kafka ou sistemas externos
- DBFS ou FileStore
- Configuração de rede
- Dados sintéticos

---

## 🔗 Recursos

- [Databricks Structured Streaming](https://docs.databricks.com/structured-streaming/index.html)
- [Delta Lake](https://docs.databricks.com/delta/index.html)
- [Unity Catalog](https://docs.databricks.com/data-governance/unity-catalog/index.html)
- [NYC Taxi Dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)